In [6]:
import pandas as pd
import itertools
import random
import json

In [7]:
valid_descr_path = "/home/jupyter/datasphere/project/valid_descr.csv"

valid_descr = pd.read_csv(valid_descr_path)

In [ ]:
valid_descr['od_excavation_ratio'].unique()

array([nan, 0.4, 0.9, 0.3, 0.5, 0.7, 0.6, 0.8, 0. ])

In [ ]:
valid_descr['vessels_ratio'].unique()

array(['2:3'], dtype=object)

Соберём словарь параметров и их всех возможных значений

Значения для показателей ratio взял из реальных данных, полученных выше

In [10]:
params_values_dict = {
    'od_color' : ["Серый", "Белый", "Бледный", "Бледно-Розовый", "Розовый", "Гиперемированный"],
    'od_monotone': ["Наблюдается", "Не наблюдается"],
    'od_size': ["Нормальный", "Больше нормы", "Меньше нормы"],
    'od_shape': ["Правильная", "Овальная", "Не правильная"],
    'od_border': ["Четкие", "Размытые"],
    'od_excavation_size': ["Нормальный", "Больше нормы", "Меньше нормы"],
    'od_excavation_location': ["В центре", "Верхний", "Нижний", "Наружный", "Внутренний", "Верхне-наружный",
                              "Верхне-внутренний", "Нижне-наружный", "Нижне-внутренний"],
    'od_vessels_location': ["В центре", "Верхний", "Нижний", "Наружный", "Внутренний", "Верхне-наружный",
                              "Верхне-внутренний", "Нижне-наружный", "Нижне-внутренний"],
    'od_excavation_ratio': [0.4, 0.9, 0.3, 0.5, 0.7, 0.6, 0.8, 0],
    'vessels_art_course': ["Нормальный", "Смещение наружу", "Смещение внутрь"],
    'vessels_art_turtuosity': ["Нормальная", "Извитые", "Прямые"],
    'vessels_art_bifurcation': ["Нормальная", "Под острым углом", "Под тупым углом"],
    'vessels_art_caliber': ["Нормальный", "Расширенный", "Суженный"],
    'vessels_vein_course': ["Нормальный", "Смещение наружу", "Смещение внутрь"],
    'vessels_vein_turtuosity': ["Нормальная", "Извитые", "Прямые"],
    'vessels_vein_bifurcation': ["Нормальная", "Под острым углом", "Под тупым углом"],
    'vessels_vein_caliber': ["Нормальный", "Расширенный", "Суженный"],
    'vessels_ratio': ["2:3"],
    'macula_macular_reflex': ["Нормальный", "Сглаженный", "Отсутсвует"],
    'macula_foveal_reflex': ["Нормальный", "Сглаженный", "Отсутсвует"],
}

Посчитаем количество уникальных комбинаций параметров

In [11]:
num_combinations = 1
for key in params_values_dict:
    num_combinations *= len(params_values_dict[key])
    
print(f"Всего комбинаций: {num_combinations}")

Всего комбинаций: 24794911296


Теперь выберем 100 случайных комбиинаций параметров, и зафиксируем получившийся набор как JSON.

Получившийся JSON будем использовать для генерации запросов для всех LLM при их сравнении. 

In [13]:
def get_random_combinations(params_dict, n=100):
    keys = list(params_dict.keys())
    result = []
    for _ in range(n):
        combo = {key: random.choice(params_dict[key]) for key in keys}
        result.append(combo)
    return result

In [14]:
# Выборка 100 случайных комбинаций
random_combos = get_random_combinations(params_values_dict, 100)

Проверяем, что получили действительно 100 различных комбинаций, потому что в подходе выше это не гарантировано random-ом

In [15]:
random_combos_df = pd.DataFrame(random_combos)

In [18]:
columns_to_check = [
    'od_color',
    'od_monotone',
    'od_size',
    'od_shape',
    'od_border',
    'od_excavation_size',
    'od_excavation_location',
    'od_excavation_ratio',
    'od_vessels_location',
    'vessels_art_course',
    'vessels_art_turtuosity',
    'vessels_art_bifurcation',
    'vessels_art_caliber',
    'vessels_vein_course',
    'vessels_vein_turtuosity',
    'vessels_vein_bifurcation',
    'vessels_vein_caliber',
    'vessels_ratio',
    'macula_macular_reflex',
    'macula_foveal_reflex',
]

In [ ]:
random_combos_df.groupby(columns_to_check).ngroups

Добавим также в комбинации параметров, где значения у артерий и вен пересекаются.

Это необходимо для того, чтобы проверить, на сколько модель способна понять входную инструкцию о том, что если есть совпадение параметров у этих двух структур, то их допускается описывать совместно. 

In [15]:
params_values_dict_av = {
    'od_color' : ["Серый", "Белый", "Бледный", "Бледно-Розовый", "Розовый", "Гиперемированный"],
    'od_monotone': ["Наблюдается", "Не наблюдается"],
    'od_size': ["Нормальный", "Больше нормы", "Меньше нормы"],
    'od_shape': ["Правильная", "Овальная", "Не правильная"],
    'od_border': ["Четкие", "Размытые"],
    'od_excavation_size': ["Нормальный", "Больше нормы", "Меньше нормы"],
    'od_excavation_location': ["В центре", "Верхний", "Нижний", "Наружный", "Внутренний", "Верхне-наружный",
                              "Верхне-внутренний", "Нижне-наружный", "Нижне-внутренний"],
    'od_vessels_location': ["В центре", "Верхний", "Нижний", "Наружный", "Внутренний", "Верхне-наружный",
                              "Верхне-внутренний", "Нижне-наружный", "Нижне-внутренний"],
    'od_excavation_ratio': [0.4, 0.9, 0.3, 0.5, 0.7, 0.6, 0.8, 0],
    'vessels_art_course': ["Смещение наружу"],
    'vessels_art_turtuosity': ["Извитые",],
    'vessels_art_bifurcation': ["Под острым углом"],
    'vessels_art_caliber': ["Нормальный"],
    'vessels_vein_course': ["Смещение наружу"],
    'vessels_vein_turtuosity': ["Извитые"],
    'vessels_vein_bifurcation': ["Под острым углом", ],
    'vessels_vein_caliber': ["Нормальный"],
    'vessels_ratio': ["2:3"],
    'macula_macular_reflex': ["Нормальный", "Сглаженный", "Отсутсвует"],
    'macula_foveal_reflex': ["Нормальный", "Сглаженный", "Отсутсвует"],
}

In [16]:
random_combos_av = get_random_combinations(params_values_dict_av, 25)

In [21]:
random_combos_av_df = pd.DataFrame(random_combos_av)
random_combos_av_df.groupby(columns_to_check).ngroups

25

In [22]:
with open("TEST_samples.json", "r", encoding="utf-8") as f:
    random_combos = json.load(f)

In [27]:
random_combos.extend(random_combos_av)

Сохраняем наши комбинации в формате JSON

In [28]:
with open("TEST_samples_v2.json", "w", encoding="utf-8") as f:
    json.dump(random_combos, f, ensure_ascii=False, indent=4)

Подгружаем наши комбинации из json документа

In [29]:
with open("TEST_samples_v2.json", "r", encoding="utf-8") as f:
    loaded_samples = json.load(f)

Теперь создадим шаблон промта для моделей. 

In [32]:
def get_prompt_v2(sample_dict):
    json_text = f"""
    {{
    "ДЗН": {{
        "Цвет": "{sample_dict["od_color"]}",
        "Монотонность": "{sample_dict["od_monotone"]}",
        "Размер": "{sample_dict["od_size"]}",
        "Форма": "{sample_dict["od_shape"]}",
        "Границы": "{sample_dict["od_border"]}",
        "Экскавация": {{
            "Размер": {sample_dict["od_excavation_size"]},
            "Сектор": {sample_dict["od_excavation_location"]}
        }},
        "Э/Д": {sample_dict["od_excavation_ratio"]},
        "Сосудистый пучок": {sample_dict["od_vessels_location"]},
    }},
    "Сосуды": {{
        "Артерии": {{
            "Ход": {sample_dict["vessels_art_course"]},
            "Извитость": {sample_dict["vessels_art_turtuosity"]},
            "Бифуркация": {sample_dict["vessels_art_bifurcation"]},
            "Калибр": {sample_dict["vessels_art_caliber"]}
        }},
        "Вены": {{
            "Ход": {sample_dict["vessels_vein_course"]},
            "Извитость": {sample_dict["vessels_vein_turtuosity"]},
            "Бифуркация": {sample_dict["vessels_vein_bifurcation"]},
            "Калибр": {sample_dict["vessels_vein_caliber"]}
        }},
        "А/В индекс": {sample_dict["vessels_ratio"]},
    }},
    "Макула": {{
        "Макулярный рефлекс": {sample_dict["macula_macular_reflex"]},
        "Фовеальный рефлекс": {sample_dict["macula_foveal_reflex"]},
    }},
    }}
    """

    prompt_tmp = f"""**Задача:**
На основе приведённого JSON-документа, содержащего описание глазного дна, сгенерируй полное текстовое описание, придерживаясь стилистики и формата примера.

**Формат:**
- Описание должно быть текстовым, структурированным и последовательным, как в приведённом примере.
- Допускается совместное описание артерий и вен, если у них одинаковые параметры (Пример 1). Если параметры разные, их необходимо описывать отдельно (Пример 2).

**Данные:**
JSON-документ:
{json_text}

**Пример 1:**
ДЗН серый, монотонность не наблюдается, нормального размера, правильной формы, границы четкие. Экскавация нормального размера, расположена в центре, э/д=0.4. Сосудистый пучок расположен центрально.  
Артерии и Вены имеют нормальный ход, нормальную извитость, бифуркация в норме и калибр нормальный, соотношение А/В = 2/3.
Макулярный рефлекс отсутствует, фовеальный рефлекс нормальный.

**Пример 2:**
ДЗН серый, границы четкие, форма правильная, размер нормальный. Экскавация нормальная, в центре. Сосудистый пучок расположен центрально.  
Артерии: ход смещён наружу, извитость нормальная, бифуркация под острым углом, калибр нормальный.
Вены: ход нормальный, извитость нормальная, бифуркация нормальная, калибр суженный.
Соотношение А/В = 2/3.
Макулярный рефлекс отсутствует, фовеальный рефлекс нормальный.

**Твоя задача:**  
Создай полное текстовое описание глазного дна по JSON-документу, следуй телеграфному стилю и примеру выше.
"""
    return prompt_tmp

Сформируем набор промтов и сохраним их в формате JSON для дальнейшего переиспользования

In [33]:
prompts_collection = [get_prompt_v2(sample) for sample in loaded_samples]

In [34]:
with open("TEST_prompts_v2.json", "w", encoding="utf-8") as f:
    json.dump(prompts_collection, f, ensure_ascii=False, indent=4)

In [35]:
prompts_collection[0]

'**Задача:**\nНа основе приведённого JSON-документа, содержащего описание глазного дна, сгенерируй полное текстовое описание, придерживаясь стилистики и формата примера.\n\n**Формат:**\n- Описание должно быть текстовым, структурированным и последовательным, как в приведённом примере.\n- Допускается совместное описание артерий и вен, если у них одинаковые параметры (Пример 1). Если параметры разные, их необходимо описывать отдельно (Пример 2).\n\n**Данные:**\nJSON-документ:\n\n    {\n    "ДЗН": {\n        "Цвет": "Розовый",\n        "Монотонность": "Не наблюдается",\n        "Размер": "Меньше нормы",\n        "Форма": "Овальная",\n        "Границы": "Четкие",\n        "Экскавация": {\n            "Размер": Нормальный,\n            "Сектор": Верхне-наружный\n        },\n        "Э/Д": 0.9,\n        "Сосудистый пучок": Внутренний,\n    },\n    "Сосуды": {\n        "Артерии": {\n            "Ход": Нормальный,\n            "Извитость": Прямые,\n            "Бифуркация": Под тупым углом,\n  

## Метрики_v1

In [3]:
df_ruadapt = pd.read_csv('/home/jupyter/datasphere/project/RuAdapt_scores.csv')
df_saiga = pd.read_csv('/home/jupyter/datasphere/project/Saiga_scores.csv')
df_tablellama = pd.read_csv('/home/jupyter/datasphere/project/TableLlama_scores.csv')
df_yandex = pd.read_csv('/home/jupyter/datasphere/project/Yandex_scores.csv')
df_mistral = pd.read_csv('/home/jupyter/datasphere/project/BioMistral_scores.csv')

In [4]:
df_yandex.head()

,Unnamed: 0,param_match_score,no_intro_phrases_score,structure_order_score,language_naturalness_score,param_match_comment,no_intro_phrases_comment,structure_order_comment,language_naturalness_comment
0,0,1,1,1,5,All parameters from the input JSON are accurat...,The text is devoid of introductory phrases and...,The textual description adheres to the specifi...,"The language used is clear, professional, and ..."
1,1,1,1,1,5,Все параметры из приведённого JSON точно соотв...,"Вводные конструкции отсутствуют, текст исключи...",Порядок описания строго соответствует ожидаемому.,"Текст составлен профессионально, в медицинском..."
2,2,1,1,1,5,Все параметры из JSON точно отображены в текст...,В тексте отсутствуют вводные конструкции или н...,Порядок описания соответствует заданной структ...,"Текст четкий, медицинский язык профессиональны..."
3,3,0,1,1,4,The parameter in 'Экскавация' with 'Размер: Ме...,No introductory phrases or unnecessary constru...,The description adheres to the expected struct...,The text maintains a professional tone with mi...
4,4,1,1,1,5,All parameters in the generated text correctly...,The text strictly contains medical observation...,The structure adheres to the prescribed order ...,The language and style are natural and appropr...


In [5]:
df_ruadapt = df_ruadapt[['param_match_score', 'no_intro_phrases_score', 'structure_order_score', 'language_naturalness_score']]
df_saiga = df_saiga[['param_match_score', 'no_intro_phrases_score', 'structure_order_score', 'language_naturalness_score']]
df_tablellama = df_tablellama[['param_match_score', 'no_intro_phrases_score', 'structure_order_score', 'language_naturalness_score']]
df_yandex = df_yandex[['param_match_score', 'no_intro_phrases_score', 'structure_order_score', 'language_naturalness_score']]
df_mistral = df_mistral[['param_match_score', 'no_intro_phrases_score', 'structure_order_score', 'language_naturalness_score']]

In [21]:
df_ruadapt.sum()

param_match_score              56
no_intro_phrases_score         74
structure_order_score          67
language_naturalness_score    425
dtype: int64

In [22]:
df_saiga.sum()

param_match_score              69
no_intro_phrases_score         80
structure_order_score          79
language_naturalness_score    449
dtype: int64

In [23]:
df_tablellama.sum()

param_match_score               0
no_intro_phrases_score          1
structure_order_score           0
language_naturalness_score    200
dtype: int64

In [24]:
df_yandex.sum()

param_match_score              92
no_intro_phrases_score        100
structure_order_score         100
language_naturalness_score    491
dtype: int64

In [25]:
df_mistral.sum()

param_match_score               2
no_intro_phrases_score         46
structure_order_score          14
language_naturalness_score    326
dtype: int64